# LG3 prepare_lg3_data demo (single notebook)

This demo mirrors `prepare_lg3_data.py` in a compact, inspectable way.
It supports:
- adjustable resample frequency
- configurable columns
- per-unit SMARTCARE handling (Auto Id used only for grouping)

We sample 5 files and display a random slice of the resulting data.


In [31]:
import os
import re
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

BASE_DIR = os.path.abspath(os.path.join('..', 'data'))
EREPORT_DIR = os.path.join(BASE_DIR, 'EREPORT')
SMARTCARE_DIR = os.path.join(BASE_DIR, 'SMARTCARE')

# Configurable inputs
RESAMPLE_FREQ = '15min'
SAMPLE_FILES = 5
SHOW_ROWS = 8
SMOOTH_WINDOW = 3

EREPORT_COLS = [
    'Capa_Cooling', 'MFR_068', 'Rop', 'Comp1 Hz_1', 'Comp1 Hz_0',
    'VAP_Entha', 'LIQ_Entha', 'cycle', 'HighP', 'LowP', 'Tcond', 'SCEEV_M'
]
SMARTCARE_COLS = ['Tset', 'Tid', 'Hid', 'Low P', 'High P', 'Power', 'Tpip_in', 'Frun']

RANDOM_SEED = 2025
np.random.seed(RANDOM_SEED)
print('Random seed:', RANDOM_SEED)


Random seed: 2025


## 1) Load 5 files and build Timestamp


In [32]:
def parse_date_from_filename(path):
    m = re.search(r'_(\d{8})\.csv$', os.path.basename(path))
    if not m:
        return None
    return pd.to_datetime(m.group(1), format='%Y%m%d')

def load_csv_with_date(path, keep_cols, require_auto_id=False):
    df = pd.read_csv(path)
    df = df.loc[:, [c for c in df.columns if not c.startswith('Unnamed')]]
    date = parse_date_from_filename(path)
    if date is not None and 'Time' in df.columns:
        df['Timestamp'] = pd.to_datetime(date.strftime('%Y-%m-%d') + ' ' + df['Time'].astype(str), errors='coerce')
    keep = ['Timestamp']
    if require_auto_id:
        keep.append('Auto Id')
    keep += [c for c in keep_cols if c in df.columns]
    return df[keep]

def load_folder(folder, keep_cols, n_files, require_auto_id=False):
    files = sorted(glob.glob(os.path.join(folder, '*.csv')))[:n_files]
    frames = [load_csv_with_date(p, keep_cols, require_auto_id=require_auto_id) for p in files]
    df = pd.concat(frames, ignore_index=True)
    df = df.dropna(subset=['Timestamp']).sort_values('Timestamp')
    return df, files

ereport_df, ereport_files = load_folder(EREPORT_DIR, EREPORT_COLS, SAMPLE_FILES)
smartcare_df, smartcare_files = load_folder(SMARTCARE_DIR, SMARTCARE_COLS, SAMPLE_FILES, require_auto_id=True)

print('EREPORT files:', [os.path.basename(p) for p in ereport_files])
print('SMARTCARE files:', [os.path.basename(p) for p in smartcare_files])
print('EREPORT raw shape:', ereport_df.shape)
print('SMARTCARE raw shape:', smartcare_df.shape)
display(ereport_df.head(5))
display(smartcare_df.head(5))


EREPORT files: ['DBG_EREPORT_20250801.csv', 'DBG_EREPORT_20250802.csv', 'DBG_EREPORT_20250803.csv', 'DBG_EREPORT_20250804.csv', 'DBG_EREPORT_20250805.csv']
SMARTCARE files: ['LOG_SMARTCARE_20250801.csv', 'LOG_SMARTCARE_20250802.csv', 'LOG_SMARTCARE_20250803.csv', 'LOG_SMARTCARE_20250804.csv', 'LOG_SMARTCARE_20250805.csv']
EREPORT raw shape: (6387, 13)
SMARTCARE raw shape: (843183, 10)


,Timestamp,Capa_Cooling,MFR_068,Rop,Comp1 Hz_1,Comp1 Hz_0,VAP_Entha,LIQ_Entha,cycle,HighP,LowP,Tcond,SCEEV_M
0,2025-08-01 13:26:07,28.1471,533.266,0.360406,36,36,429.628,239.604,1,2483,823,42.7909,63
1,2025-08-01 13:28:32,27.4312,517.248,0.360406,36,36,430.021,239.094,1,2524,794,43.4636,64
2,2025-08-01 13:29:32,26.5758,501.673,0.360406,34,34,429.812,239.096,1,2516,823,43.3424,64
3,2025-08-01 13:30:32,26.7764,509.349,0.360406,34,34,429.542,240.283,1,2451,836,42.2500,64
4,2025-08-01 13:31:32,26.6505,503.651,0.360406,34,34,429.620,239.120,1,2418,826,41.6909,64


,Timestamp,Auto Id,Tset,Tid,Hid,Low P,High P,Power,Tpip_in,Frun
0,2025-08-01 13:25:07,1,21.0,27.3,47.0,839.0,2491.0,0.0,6.0,0
1,2025-08-01 13:25:07,2,21.0,28.0,54.0,839.0,2491.0,437.0,9.4,5
2,2025-08-01 13:25:07,3,20.0,28.4,42.0,839.0,2491.0,0.0,22.5,0
3,2025-08-01 13:25:07,4,21.0,27.3,42.0,839.0,2491.0,1845.0,9.4,5
4,2025-08-01 13:25:07,5,21.0,22.3,48.0,839.0,2491.0,1845.0,8.7,5


## 2) Resample EREPORT (base)


In [33]:
def to_numeric(df, cols):
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors='coerce')
    return df

ereport_df = to_numeric(ereport_df, EREPORT_COLS).set_index('Timestamp')
ereport_base = ereport_df[EREPORT_COLS].resample(RESAMPLE_FREQ).mean()

print('EREPORT base shape:', ereport_base.shape)
display(ereport_base.head(5))


EREPORT base shape: (427, 12)


,Capa_Cooling,MFR_068,Rop,Comp1 Hz_1,Comp1 Hz_0,VAP_Entha,LIQ_Entha,cycle,HighP,LowP,Tcond,SCEEV_M
Timestamp,,,,,,,,,,,,
2025-08-01 13:15:00,27.384700,517.395667,0.360406,35.333333,35.333333,429.820333,239.264667,1.0,2507.666667,813.333333,43.198967,63.666667
2025-08-01 13:30:00,27.492980,525.600400,0.360406,35.200000,35.200000,429.427600,240.927400,1.0,2479.133333,831.733333,42.724440,64.200000
2025-08-01 13:45:00,34.202553,650.481467,0.438579,44.000000,44.000000,428.996067,239.653667,1.0,2476.333333,811.733333,42.668947,65.800000
2025-08-01 14:00:00,20.925387,399.539533,0.250762,25.266667,31.600000,428.091667,239.625267,1.0,2354.800000,791.266667,40.637227,61.666667
2025-08-01 14:15:00,13.491237,261.866267,0.234518,0.000000,35.600000,428.493200,242.295800,1.0,2425.666667,813.533333,41.822280,60.133333


## 3) SMARTCARE per-unit resample + smoothing
Auto Id is used only for grouping (not kept as a column).


In [34]:
smartcare_df = to_numeric(smartcare_df, SMARTCARE_COLS)
unit_ids = sorted(smartcare_df['Auto Id'].dropna().unique().tolist())
print('Auto Id count:', len(unit_ids))
print('Auto Id sample:', unit_ids[:5])

show_units = unit_ids[:3]
for unit_id in show_units:
    unit = smartcare_df[smartcare_df['Auto Id'] == unit_id].copy()
    unit = unit.set_index('Timestamp')[SMARTCARE_COLS].resample(RESAMPLE_FREQ).mean()
    if SMOOTH_WINDOW > 1:
        unit = unit.rolling(window=SMOOTH_WINDOW, min_periods=1).mean()

    print(f'Unit {unit_id} shape:', unit.shape)
    display(unit.head(5))


Auto Id count: 11
Auto Id sample: [1, 2, 3, 4, 5]
Unit 1 shape: (427, 8)


,Tset,Tid,Hid,Low P,High P,Power,Tpip_in,Frun
Timestamp,,,,,,,,
2025-08-01 13:15:00,21.0,27.3,47.000000,816.181818,2492.545455,0.0,5.543182,0.0
2025-08-01 13:30:00,21.0,27.3,46.875000,823.560354,2486.561616,0.0,5.831035,0.0
2025-08-01 13:45:00,21.0,27.3,46.642593,819.649495,2481.848485,0.0,5.798468,0.0
2025-08-01 14:00:00,21.0,27.3,46.596296,811.275926,2438.101852,0.0,5.497037,0.0
2025-08-01 14:15:00,21.0,27.3,46.559259,805.483333,2419.520370,0.0,5.274444,0.0


Unit 2 shape: (427, 8)


,Tset,Tid,Hid,Low P,High P,Power,Tpip_in,Frun
Timestamp,,,,,,,,
2025-08-01 13:15:00,21.000000,28.000000,54.204545,816.181818,2492.545455,451.204545,8.902273,5.000000
2025-08-01 13:30:00,21.000000,27.999167,54.116162,823.560354,2486.561616,452.052273,9.191692,5.000000
2025-08-01 13:45:00,21.000000,27.797222,53.938552,819.649495,2481.848485,480.807071,9.179461,5.000000
2025-08-01 14:00:00,20.990741,27.570556,53.620370,811.275926,2438.101852,423.877778,9.958519,4.007407
2025-08-01 14:15:00,20.990741,27.547778,53.529630,805.483333,2419.520370,272.911111,12.381111,2.340741


Unit 3 shape: (427, 8)


,Tset,Tid,Hid,Low P,High P,Power,Tpip_in,Frun
Timestamp,,,,,,,,
2025-08-01 13:15:00,20.0,28.400000,42.000000,816.181818,2492.545455,0.0,22.500000,0.0
2025-08-01 13:30:00,20.0,28.400000,42.000000,823.560354,2486.561616,0.0,22.500000,0.0
2025-08-01 13:45:00,20.0,28.400000,41.988889,819.649495,2481.848485,0.0,22.488889,0.0
2025-08-01 14:00:00,20.0,28.386667,41.962963,811.275926,2438.101852,0.0,22.404444,0.0
2025-08-01 14:15:00,20.0,28.270370,41.962963,805.483333,2419.520370,0.0,22.359444,0.0


## 4) Merge base with per-unit SMARTCARE (demo only)
Shows what a per-unit merged dataset looks like.


In [35]:
for unit_id in show_units:
    unit = smartcare_df[smartcare_df['Auto Id'] == unit_id].copy()
    unit = unit.set_index('Timestamp')[SMARTCARE_COLS].resample(RESAMPLE_FREQ).mean()
    if SMOOTH_WINDOW > 1:
        unit = unit.rolling(window=SMOOTH_WINDOW, min_periods=1).mean()
    merged = ereport_base.join(unit, how='inner')
    print(f'Merged unit {unit_id} shape:', merged.shape)
    display(merged.head(5))


Merged unit 1 shape: (427, 20)


,Capa_Cooling,MFR_068,Rop,Comp1 Hz_1,Comp1 Hz_0,VAP_Entha,LIQ_Entha,cycle,HighP,LowP,Tcond,SCEEV_M,Tset,Tid,Hid,Low P,High P,Power,Tpip_in,Frun
Timestamp,,,,,,,,,,,,,,,,,,,,
2025-08-01 13:15:00,27.384700,517.395667,0.360406,35.333333,35.333333,429.820333,239.264667,1.0,2507.666667,813.333333,43.198967,63.666667,21.0,27.3,47.000000,816.181818,2492.545455,0.0,5.543182,0.0
2025-08-01 13:30:00,27.492980,525.600400,0.360406,35.200000,35.200000,429.427600,240.927400,1.0,2479.133333,831.733333,42.724440,64.200000,21.0,27.3,46.875000,823.560354,2486.561616,0.0,5.831035,0.0
2025-08-01 13:45:00,34.202553,650.481467,0.438579,44.000000,44.000000,428.996067,239.653667,1.0,2476.333333,811.733333,42.668947,65.800000,21.0,27.3,46.642593,819.649495,2481.848485,0.0,5.798468,0.0
2025-08-01 14:00:00,20.925387,399.539533,0.250762,25.266667,31.600000,428.091667,239.625267,1.0,2354.800000,791.266667,40.637227,61.666667,21.0,27.3,46.596296,811.275926,2438.101852,0.0,5.497037,0.0
2025-08-01 14:15:00,13.491237,261.866267,0.234518,0.000000,35.600000,428.493200,242.295800,1.0,2425.666667,813.533333,41.822280,60.133333,21.0,27.3,46.559259,805.483333,2419.520370,0.0,5.274444,0.0


Merged unit 2 shape: (427, 20)


,Capa_Cooling,MFR_068,Rop,Comp1 Hz_1,Comp1 Hz_0,VAP_Entha,LIQ_Entha,cycle,HighP,LowP,Tcond,SCEEV_M,Tset,Tid,Hid,Low P,High P,Power,Tpip_in,Frun
Timestamp,,,,,,,,,,,,,,,,,,,,
2025-08-01 13:15:00,27.384700,517.395667,0.360406,35.333333,35.333333,429.820333,239.264667,1.0,2507.666667,813.333333,43.198967,63.666667,21.000000,28.000000,54.204545,816.181818,2492.545455,451.204545,8.902273,5.000000
2025-08-01 13:30:00,27.492980,525.600400,0.360406,35.200000,35.200000,429.427600,240.927400,1.0,2479.133333,831.733333,42.724440,64.200000,21.000000,27.999167,54.116162,823.560354,2486.561616,452.052273,9.191692,5.000000
2025-08-01 13:45:00,34.202553,650.481467,0.438579,44.000000,44.000000,428.996067,239.653667,1.0,2476.333333,811.733333,42.668947,65.800000,21.000000,27.797222,53.938552,819.649495,2481.848485,480.807071,9.179461,5.000000
2025-08-01 14:00:00,20.925387,399.539533,0.250762,25.266667,31.600000,428.091667,239.625267,1.0,2354.800000,791.266667,40.637227,61.666667,20.990741,27.570556,53.620370,811.275926,2438.101852,423.877778,9.958519,4.007407
2025-08-01 14:15:00,13.491237,261.866267,0.234518,0.000000,35.600000,428.493200,242.295800,1.0,2425.666667,813.533333,41.822280,60.133333,20.990741,27.547778,53.529630,805.483333,2419.520370,272.911111,12.381111,2.340741


Merged unit 3 shape: (427, 20)


,Capa_Cooling,MFR_068,Rop,Comp1 Hz_1,Comp1 Hz_0,VAP_Entha,LIQ_Entha,cycle,HighP,LowP,Tcond,SCEEV_M,Tset,Tid,Hid,Low P,High P,Power,Tpip_in,Frun
Timestamp,,,,,,,,,,,,,,,,,,,,
2025-08-01 13:15:00,27.384700,517.395667,0.360406,35.333333,35.333333,429.820333,239.264667,1.0,2507.666667,813.333333,43.198967,63.666667,20.0,28.400000,42.000000,816.181818,2492.545455,0.0,22.500000,0.0
2025-08-01 13:30:00,27.492980,525.600400,0.360406,35.200000,35.200000,429.427600,240.927400,1.0,2479.133333,831.733333,42.724440,64.200000,20.0,28.400000,42.000000,823.560354,2486.561616,0.0,22.500000,0.0
2025-08-01 13:45:00,34.202553,650.481467,0.438579,44.000000,44.000000,428.996067,239.653667,1.0,2476.333333,811.733333,42.668947,65.800000,20.0,28.400000,41.988889,819.649495,2481.848485,0.0,22.488889,0.0
2025-08-01 14:00:00,20.925387,399.539533,0.250762,25.266667,31.600000,428.091667,239.625267,1.0,2354.800000,791.266667,40.637227,61.666667,20.0,28.386667,41.962963,811.275926,2438.101852,0.0,22.404444,0.0
2025-08-01 14:15:00,13.491237,261.866267,0.234518,0.000000,35.600000,428.493200,242.295800,1.0,2425.666667,813.533333,41.822280,60.133333,20.0,28.270370,41.962963,805.483333,2419.520370,0.0,22.359444,0.0


## 5) Sync check (aligned timestamps)
Show the same timestamp window for EREPORT, SMARTCARE (unit), and the merged table.


In [52]:
unit_id = show_units[0]
unit = smartcare_df[smartcare_df['Auto Id'] == unit_id].copy()
unit = unit.set_index('Timestamp')[SMARTCARE_COLS].resample(RESAMPLE_FREQ).mean()
if SMOOTH_WINDOW > 1:
    unit = unit.rolling(window=SMOOTH_WINDOW, min_periods=1).mean()

merged = ereport_base.join(unit, how='inner')

if len(merged) > 5:
    start = np.random.randint(0, len(merged) - 5)
    demo_merged = merged.iloc[start:start + 5]
else:
    demo_merged = merged

# Show the same timestamps from each source
ts_window = demo_merged.index
demo_ereport = ereport_base.loc[ts_window]
demo_smartcare = unit.loc[ts_window]

print(f'Unit {unit_id} - EREPORT sample')
display(demo_ereport)
print(f'Unit {unit_id} - SMARTCARE sample')
display(demo_smartcare)
print(f'Unit {unit_id} - MERGED sample')
display(demo_merged)


Unit 1 - EREPORT sample


,Capa_Cooling,MFR_068,Rop,Comp1 Hz_1,Comp1 Hz_0,VAP_Entha,LIQ_Entha,cycle,HighP,LowP,Tcond,SCEEV_M
Timestamp,,,,,,,,,,,,
2025-08-03 22:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1588.200000,1557.600000,25.867880,20.0
2025-08-03 22:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1593.000000,1562.000000,25.969700,20.0
2025-08-03 23:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1593.000000,1564.400000,25.969700,20.0
2025-08-03 23:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1600.466667,1571.666667,26.128087,20.0
2025-08-03 23:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1605.266667,1575.800000,26.229907,20.0


Unit 1 - SMARTCARE sample


,Tset,Tid,Hid,Low P,High P,Power,Tpip_in,Frun
Timestamp,,,,,,,,
2025-08-03 22:30:00,21.0,31.800000,66.000000,1557.040741,1587.553704,0.0,29.300000,0.0
2025-08-03 22:45:00,21.0,31.842222,66.000000,1560.027778,1590.866667,0.0,29.300000,0.0
2025-08-03 23:00:00,21.0,31.963704,66.085185,1561.435185,1591.503704,0.0,29.300000,0.0
2025-08-03 23:15:00,21.0,32.097037,66.418519,1566.103704,1595.814815,0.0,29.366667,0.0
2025-08-03 23:30:00,21.0,32.188148,66.751852,1570.727778,1599.948148,0.0,29.496296,0.0


Unit 1 - MERGED sample


,Capa_Cooling,MFR_068,Rop,Comp1 Hz_1,Comp1 Hz_0,VAP_Entha,LIQ_Entha,cycle,HighP,LowP,Tcond,SCEEV_M,Tset,Tid,Hid,Low P,High P,Power,Tpip_in,Frun
Timestamp,,,,,,,,,,,,,,,,,,,,
2025-08-03 22:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1588.200000,1557.600000,25.867880,20.0,21.0,31.800000,66.000000,1557.040741,1587.553704,0.0,29.300000,0.0
2025-08-03 22:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1593.000000,1562.000000,25.969700,20.0,21.0,31.842222,66.000000,1560.027778,1590.866667,0.0,29.300000,0.0
2025-08-03 23:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1593.000000,1564.400000,25.969700,20.0,21.0,31.963704,66.085185,1561.435185,1591.503704,0.0,29.300000,0.0
2025-08-03 23:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1600.466667,1571.666667,26.128087,20.0,21.0,32.097037,66.418519,1566.103704,1595.814815,0.0,29.366667,0.0
2025-08-03 23:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1605.266667,1575.800000,26.229907,20.0,21.0,32.188148,66.751852,1570.727778,1599.948148,0.0,29.496296,0.0
